# Neural Network Evaluation 

Classification task not supported yet 

See: `Events.add_prediction`, `LoadableModel.categorical_estimation` and last section


In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = '1'

In [ ]:
import time
from pathlib import Path
from tqdm import tqdm

import pandas as pd

from gerumo.data.dataset import describe_dataset
from gerumo.data.generators import build_generator
from gerumo.utils.engine import (
    setup_cfg, setup_environment, setup_experiment, setup_model, load_model, build_dataset, get_dataset_name
)
from gerumo.utils.structures import Event, Task
from gerumo.models.base import build_model
from gerumo.visualization.metrics import training_history


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
args = dotdict()

## Select experiment

In [ ]:
# Select a trained model directory
#args['config_file'] = '/home/asuka/projects/gerumo2/tools/output/20220224_101409_cnn_mst_regression_concat_first'
args['config_file'] = '/home/asuka/projects/gerumo2/tools/output/20220224_154553_cnn_mst_regression_concat_last'
#args['config_file'] = '/home/asuka/projects/gerumo2/tools/output/20220224_045708_cnn_mst_regression_pe_first'
#args['config_file'] = '/home/asuka/projects/gerumo2/tools/output/20220223_234309_cnn_mst_regression_pe_last'

# Select the best batch
args['epoch'] = -1

# Use the validation datasets for evaluation
args['use_validation'] = True

# Select a test datasets (on axis/off axis)
args['opts'] = [
    #'DATASETS.TEST.EVENTS', ''
    #'DATASETS.TEST.TELESCOPES', ''
]

## Setup

In [ ]:
# Find the config.yml
if os.path.isdir(args.config_file):
    args.config_file = os.path.join(args.config_file, 'config.yml')
# Load the configurations
cfg = setup_cfg(args)
output_dir, evaluation_dir = setup_experiment(cfg, training=False)
logger = setup_environment(cfg)

## Load evaluation dataset

In [ ]:
# Setup evaluation datasets directory
if args.use_validation:
    evaluation_dataset_name = get_dataset_name(cfg, 'validation')
else:
    evaluation_dataset_name = get_dataset_name(cfg, 'test')
evaluation_dir = evaluation_dir / evaluation_dataset_name
evaluation_dir.mkdir(exist_ok=True)

# Build evaluation dataset
evaluation_dataset = build_dataset(cfg, evaluation_dataset_name)
describe_dataset(evaluation_dataset, logger, save_to=evaluation_dir / 'description.txt')

## Build generator

In [ ]:
evaluation_generator = build_generator(cfg, evaluation_dataset)

## Load model

In [ ]:
#%%capture
input_shape = evaluation_generator.get_input_shape()
model = build_model(cfg, input_shape)
#model.summary()
model = load_model(model, evaluation_generator, output_dir, args.epoch)

## Start evaluation

In [ ]:
events = []
for X, event_true in tqdm(evaluation_generator):
    y_pred = model(X)
    events += Event.add_prediction_list(event_true, y_pred, model.task)
evaluation_results = Event.list_to_dataframe(events)
evaluation_results.to_csv(evaluation_dir / 'results.csv')

In [ ]:
evaluation_results

## Plot results

In [ ]:
from gerumo.visualization import metrics

In [ ]:
evaluation_results = pd.read_csv(evaluation_dir / 'results.csv')
evaluation_results

In [ ]:
if Task[cfg.MODEL.TASK] is Task.REGRESSION:
    # Target Regression
    targets = [t.split('_')[1] for t in cfg.OUTPUT.REGRESSION.TARGETS]
    metrics.targets_regression(evaluation_results, targets, save_to=evaluation_dir)
    # Resolution
    metrics.reconstruction_resolution(evaluation_results, targets, ylim=(0, 2), save_to=evaluation_dir)
    # Theta2 distribution
    metrics.theta2_distribution(evaluation_results, targets, save_to=evaluation_dir)

In [ ]:
if model.task is Task.CLASSIFICATION:
    pass